In [1]:
from dataclasses import dataclass, field
import copy
from enum import Enum, auto
import random
import pandas as pd

import blackjack as bj
from blackjack import Hand, make_hand

In [2]:
h = Hand()
h

Hand(score=0, soft=False, cards=[], doubled=False)

In [3]:
h.add_card(6) # start a hand with a 6

Hand(score=6, soft=False, cards=[6], doubled=False)

In [4]:
h.add_card(11) # show that J (11) counts as 10 points

Hand(score=16, soft=False, cards=[6, 11], doubled=False)

In [5]:
h.add_card(7), bj.is_busted(h) # bust (show that 23 is counted as 22)

(Hand(score=22, soft=False, cards=[6, 11, 7], doubled=False), True)

In [6]:
# OK, now let's try another hand with aces
h = Hand()
h.add_card(1) # should be a soft 11

Hand(score=11, soft=True, cards=[1], doubled=False)

In [7]:
h.add_card(1) # two aces: should be a soft 12

Hand(score=12, soft=True, cards=[1, 1], doubled=False)

In [8]:
h.add_card(11) # two aces and a J: should be a hard 12

Hand(score=12, soft=False, cards=[1, 1, 11], doubled=False)

In [9]:
# OK, now let's try another hand with a blackjack
h = Hand()
h.add_card(1) # should be a soft 11
h.add_card(10)
h, bj.is_blackjack(h)

(Hand(score=21, soft=True, cards=[1, 10], doubled=False), True)

## Now define gameplay and strategy

In [10]:

# TODO I might want a Flag class later, to provide a set of possible Actions
class Action(Enum):
    STAND = auto()
    HIT = auto()
    DOUBLE = auto()
    #SPLIT = auto()
    
    

In [11]:
# Most simple/conservative strategy imaginable:
def strat_nobust(hand, dealer):
    if hand.score > 11:
        return Action.STAND
    else:
        return Action.HIT
        
strat_nobust.name = 'strat_nobust'

In [12]:
# Dealer strategy
def strat_dealer(hand, dealer):
    if hand.score < 17:
        return Action.HIT
    # TODO handle soft hands
    else:
        return Action.STAND
    
strat_dealer.name='strat_dealer'
        

In [13]:
class HandOutcome(Enum):
    WIN = 1
    LOSE = -1
    WIN_DOUBLE = 2
    LOSE_DOUBLE = -2
    PUSH = 0
    BLACKJACK = 1.5

In [14]:
# Deck; completely random (i.e., infinite) for now


def deal_card():
    return random.randrange(13)+1

In [15]:
[deal_card() for _ in range(10)]

[9, 3, 2, 6, 4, 13, 4, 4, 8, 8]

In [16]:
# return the final hand after playing
def player_play_hand(strategy, hand, dealer, deck):
    while True:
        decision = strategy(hand, dealer)
        if decision == Action.STAND:
            return hand
        if decision == Action.HIT:
            hand.add_card(deck())
            if bj.is_busted(hand):
                return hand
        if decision == Action.DOUBLE:
            hand.doubled = True
            hand.add_card(deck())
            return hand



In [17]:
def player_hand_outcome(player_hand, dealer_hand):
    # First compute the initial outcome, then double it if necessary for a double-down
    def initial_outcome():
        if bj.is_blackjack(player_hand):
            if bj.is_blackjack(dealer_hand):
                return HandOutcome.PUSH
            else:
                return HandOutcome.BLACKJACK
        if bj.is_busted(player_hand) or bj.is_blackjack(dealer_hand):
            return HandOutcome.LOSE
        if bj.is_busted(dealer_hand):
            return HandOutcome.WIN
        if player_hand.score > dealer_hand.score:
            return HandOutcome.WIN
        if player_hand.score == dealer_hand.score:
            return HandOutcome.PUSH
        if player_hand.score < dealer_hand.score:
            return HandOutcome.LOSE

    outcome = initial_outcome()

    outcome_doubler = {HandOutcome.WIN: HandOutcome.WIN_DOUBLE, HandOutcome.LOSE: HandOutcome.LOSE_DOUBLE}

    if player_hand.doubled:
        outcome = outcome_doubler.get(outcome) or outcome
    return outcome
        


In [18]:
def get_strat_name(strat):
    if hasattr(strat, 'name'):
        return strat.name
    return repr(strat)

In [19]:
# Goal is to evaluate strategies, so make comparisons simple

# For each round:
# Multiple players all play with a copy of the same starting hand
# Each player has a strategy that they play
# Dealer plays dealer strategy

# For now, we're using an infinite deck and strategies without knowledge, so
# the interaction of players/strategies should be a wash

def deal_one_round():
    hand_p = Hand()
    hand_d = Hand()

    hand_p.add_card(deal_card())
    hand_d.add_card(deal_card())
    hand_p.add_card(deal_card())
    
    dealer_hole_card = deal_card()
    
    return hand_p, hand_d, dealer_hole_card


# Play multiple strategies on one starting point
def complete_one_round(strats, player_hand, dealer_hand, dealer_hole_card):
    hand_p = copy.deepcopy(player_hand)
    hand_d = copy.deepcopy(dealer_hand)
    
    # represent each player as a hand and a strategy
    players = [(player_play_hand(strat, copy.deepcopy(hand_p), hand_d, deal_card), get_strat_name(strat)) for strat in strats]
    
    # dealer
    player_play_hand(strat_dealer, hand_d.add_card(dealer_hole_card), None, deal_card)
    
    return [(strat, hand_p, hand_d, player_hand_outcome(hand_p, hand_d)) for (hand_p, strat) in players]

    
def play_one_round(strats):
    hand_p, hand_d, dealer_hole_card = deal_one_round()
    return complete_one_round(strats, hand_p, hand_d, dealer_hole_card)

play_one_round([strat_nobust, strat_nobust])

[('strat_nobust',
  Hand(score=15, soft=False, cards=[5, 10], doubled=False),
  Hand(score=17, soft=False, cards=[7, 13], doubled=False),
  <HandOutcome.LOSE: -1>),
 ('strat_nobust',
  Hand(score=15, soft=False, cards=[5, 10], doubled=False),
  Hand(score=17, soft=False, cards=[7, 13], doubled=False),
  <HandOutcome.LOSE: -1>)]

## Aggregate and summarize the data from the simulations

In [20]:

def generate_row_from_player(player):
    (strat, hand_p, hand_d, outcome) = player
    return {'strategy': strat, 'hand_start': hand_p.cards[:2], 'dealer_card': hand_d.cards[0], 'hand_end': hand_p.cards, 'dealer_hand': hand_d.cards, 'outcome': outcome}

def generate_rows_from_round(r):
    return [generate_row_from_player(player) for player in r]


generate_rows_from_round(play_one_round([strat_nobust, strat_dealer]))


[{'strategy': 'strat_nobust',
  'hand_start': [4, 8],
  'dealer_card': 11,
  'hand_end': [4, 8],
  'dealer_hand': [11, 13],
  'outcome': <HandOutcome.LOSE: -1>},
 {'strategy': 'strat_dealer',
  'hand_start': [4, 8],
  'dealer_card': 11,
  'hand_end': [4, 8, 6],
  'dealer_hand': [11, 13],
  'outcome': <HandOutcome.LOSE: -1>}]

In [21]:
def run_n_sim_trials(strats, n):
    sims = [generate_rows_from_round(play_one_round(strats)) for _ in range(n)]
    results = pd.DataFrame([player for round in sims for player in round])
    results['outcome_value'] = results['outcome'].apply(lambda x: x.value)
    results['outcome_name'] = results['outcome'].apply(lambda x: str(x)[12:])
    return results

def summarize_totals(sims):
    def outcome_name(x): return x.head(1) # The function name will be used as the column name
    outcome_counts = sims.groupby(['strategy', 'outcome_value'])['outcome_name'].agg([len, outcome_name])
    outcome_summary = outcome_counts.reset_index().set_index('strategy').drop(['outcome_value'], axis=1).pivot(columns=['outcome_name'])

    # The empty cells are NaNs; fill the NaNs and convert back to int
    for col in outcome_summary.columns:
        outcome_summary[col] = outcome_summary[col].fillna(0).apply(int)
        
    outcome_summary['mean_outcome'] = sims.groupby('strategy')['outcome_value'].mean()
    
    return outcome_summary


sim_results = run_n_sim_trials([strat_nobust, strat_dealer], 1000)
sim_results, summarize_totals(sim_results)

(          strategy hand_start  dealer_card    hand_end       dealer_hand  \
 0     strat_nobust    [12, 5]            4     [12, 5]     [4, 5, 5, 13]   
 1     strat_dealer    [12, 5]            4  [12, 5, 5]     [4, 5, 5, 13]   
 2     strat_nobust   [12, 10]           10    [12, 10]        [10, 2, 7]   
 3     strat_dealer   [12, 10]           10    [12, 10]        [10, 2, 7]   
 4     strat_nobust    [6, 11]            8     [6, 11]            [8, 9]   
 ...            ...        ...          ...         ...               ...   
 1995  strat_dealer    [1, 11]            3     [1, 11]  [3, 3, 6, 2, 13]   
 1996  strat_nobust    [8, 10]           13     [8, 10]       [13, 3, 13]   
 1997  strat_dealer    [8, 10]           13     [8, 10]       [13, 3, 13]   
 1998  strat_nobust    [9, 12]            2     [9, 12]      [2, 2, 9, 7]   
 1999  strat_dealer    [9, 12]            2     [9, 12]      [2, 2, 9, 7]   
 
                     outcome  outcome_value outcome_name  
 0           Ha

In [22]:
def strat_simple(hand, dealer):
    if hand.score == 11:  return Action.DOUBLE
    if hand.score >= 17:  return Action.STAND
    if hand.score <= 11:  return Action.HIT
    if dealer.score in (range(3,7)):  return Action.STAND
    else:  return Action.HIT
        
strat_simple.name = 'simple'


sims = run_n_sim_trials([strat_simple], 1000)
sims.head(10), summarize_totals(sims)

(  strategy hand_start  dealer_card     hand_end     dealer_hand  \
 0   simple     [7, 3]            8   [7, 3, 12]          [8, 9]   
 1   simple     [4, 3]           11   [4, 3, 11]      [11, 3, 8]   
 2   simple     [4, 7]            6    [4, 7, 2]       [6, 9, 8]   
 3   simple     [6, 5]            9   [6, 5, 12]          [9, 8]   
 4   simple    [13, 3]           13  [13, 3, 11]      [13, 6, 2]   
 5   simple    [11, 2]            4      [11, 2]     [4, 11, 11]   
 6   simple    [6, 11]            2   [6, 11, 7]      [2, 6, 12]   
 7   simple    [6, 12]            2   [6, 12, 6]  [2, 11, 2, 11]   
 8   simple     [8, 6]           10   [8, 6, 11]        [10, 11]   
 9   simple     [8, 4]           13    [8, 4, 5]      [13, 3, 4]   
 
                   outcome  outcome_value outcome_name  
 0         HandOutcome.WIN            1.0          WIN  
 1        HandOutcome.LOSE           -1.0         LOSE  
 2  HandOutcome.WIN_DOUBLE            2.0   WIN_DOUBLE  
 3  HandOutcome.WIN_DO

In [23]:
summarize_totals(run_n_sim_trials([strat_dealer, strat_nobust, strat_simple], 10000))


len                                         mean_outcome
outcome_name BLACKJACK  LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                                   
simple             488  4598         255  841  3506        312      -0.0246
strat_dealer       488  4881           0  959  3672          0      -0.0477
strat_nobust       488  5155           0  618  3739          0      -0.0684

## Simulate specific situations to determine strategy

In [24]:
# 12 vs deuce

hand_p = bj.make_hand([5, 7])
hand_d = bj.make_hand([2])

hand_p, hand_d

(Hand(score=12, soft=False, cards=[5, 7], doubled=False),
 Hand(score=2, soft=False, cards=[2], doubled=False))

In [25]:
# Implement a strategy that composes conditions and actions

def cond_12_2(hand, dealer):
    return hand.score == 12 and dealer.score == 2 and not hand.soft
conditions = [(cond_12_2, Action.STAND)]

def generate_strat_conditional(strat_base, conditions):
    def strat_cond(hand, dealer):
        for (condition, action) in conditions:
            if condition(hand, dealer): return action
        return strat_base(hand, dealer)
    strat_cond.name = 'strat_cond'
    return strat_cond
    
strat_cond = generate_strat_conditional(strat_simple, conditions)

In [26]:
def gen_cond_strategies(strat_base, condition, actions):
    def gen_strat_action(strat_base, condition, action):
        strat = generate_strat_conditional(strat_base, [(condition, action)])
        strat.name = repr(action)
        return strat
    
    strats = [gen_strat_action(strat_base, condition, a) for a in actions]
    return strats
    



strats = gen_cond_strategies(strat_simple, cond_12_2, [Action.HIT, Action.STAND, Action.DOUBLE])
strats

[<function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>]

In [27]:
complete_one_round(strats, hand_p, hand_d, deal_card())

[('<Action.HIT: 2>',
  Hand(score=18, soft=False, cards=[5, 7, 6], doubled=False),
  Hand(score=22, soft=False, cards=[2, 4, 2, 8, 7], doubled=False),
  <HandOutcome.WIN: 1>),
 ('<Action.STAND: 1>',
  Hand(score=12, soft=False, cards=[5, 7], doubled=False),
  Hand(score=22, soft=False, cards=[2, 4, 2, 8, 7], doubled=False),
  <HandOutcome.WIN: 1>),
 ('<Action.DOUBLE: 3>',
  Hand(score=21, soft=False, cards=[5, 7, 9], doubled=True),
  Hand(score=22, soft=False, cards=[2, 4, 2, 8, 7], doubled=False),
  <HandOutcome.WIN_DOUBLE: 2>)]

In [28]:
# TODO reduce duplication with run_n_sim_trials
def run_n_sim_trials_from_state(strats, hand_p, hand_d, n):
    sims = [generate_rows_from_round(complete_one_round(strats, hand_p, hand_d, deal_card())) for _ in range(n)]
    results = pd.DataFrame([player for round in sims for player in round])
    results['outcome_value'] = results['outcome'].apply(lambda x: x.value)
    results['outcome_name'] = results['outcome'].apply(lambda x: str(x)[12:])
    return results

sims = run_n_sim_trials_from_state(strats, hand_p, hand_d, 10000)
summarize_totals(sims)

len                                   mean_outcome
outcome_name        LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                               
<Action.DOUBLE: 3>     0        5985  513     0       3502      -0.4966
<Action.HIT: 2>     6070           0  591  3339          0      -0.2731
<Action.STAND: 1>   6345           0    0  3655          0      -0.2690

In [29]:
# Soft 12 vs 2
hand_12s = bj.make_hand([1, 1])

hand_12s, hand_d

(Hand(score=12, soft=True, cards=[1, 1], doubled=False),
 Hand(score=2, soft=False, cards=[2], doubled=False))

In [30]:
def cond_12s_2(hand, dealer):
    return hand.score == 12 and dealer.score == 2 and hand.soft

strats = gen_cond_strategies(strat_simple, cond_12s_2, [Action.HIT, Action.STAND, Action.DOUBLE])
strats

[<function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>,
 <function __main__.generate_strat_conditional.<locals>.strat_cond(hand, dealer)>]

In [31]:
sims = run_n_sim_trials_from_state(strats+[strat_simple], hand_12s, hand_d, 10000)
summarize_totals(sims)

len                                   mean_outcome
outcome_name        LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                               
<Action.DOUBLE: 3>     0        6420    0     0       3580      -0.5680
<Action.HIT: 2>     4829         459  783  3181        748      -0.1070
<Action.STAND: 1>   6420           0    0  3580          0      -0.2840
simple              4863         426  817  3160        734      -0.1087

In [32]:
summarize_totals(run_n_sim_trials_from_state([strat_simple, strat_cond], hand_p, hand_d, 100*1000))

len              mean_outcome
outcome_name   LOSE  PUSH    WIN             
strategy                                     
simple        60776  6618  32606     -0.28170
strat_cond    63799     0  36201     -0.27598

In [33]:
# soft 13 vs 6
def cond_13s_6(hand, dealer):
    return hand.score == 13 and dealer.score == 6 and hand.soft



strats = gen_cond_strategies(strat_simple, cond_13s_6, [Action.HIT, Action.STAND, Action.DOUBLE])
sims = run_n_sim_trials_from_state(strats+[strat_simple], bj.make_hand([1, 2]), bj.make_hand([6]), 100000)
summarize_totals(sims)

len                                     mean_outcome
outcome_name         LOSE LOSE_DOUBLE  PUSH    WIN WIN_DOUBLE             
strategy                                                                  
<Action.DOUBLE: 3>      0       57595     0      0      42405     -0.30380
<Action.HIT: 2>     45087        3265  3065  41728       6855      0.03821
<Action.STAND: 1>   57595           0     0  42405          0     -0.15190
simple              57595           0     0  42405          0     -0.15190

In [34]:
# 10 v 6
def cond_10_6(hand, dealer):
    return hand.score == 10 and dealer.score == 6 and not hand.soft

strats = gen_cond_strategies(strat_simple, cond_10_6, [Action.HIT, Action.STAND, Action.DOUBLE])
sims = run_n_sim_trials_from_state(strats+[strat_simple], make_hand([5, 5]), make_hand([6]), 10000)
summarize_totals(sims)

len                                   mean_outcome
outcome_name        LOSE LOSE_DOUBLE PUSH   WIN WIN_DOUBLE             
strategy                                                               
<Action.DOUBLE: 3>     0        3086  703     0       6211       0.6250
<Action.HIT: 2>     3164           0  686  6150          0       0.2986
<Action.STAND: 1>   5668           0    0  4332          0      -0.1336
simple              3152           0  710  6138          0       0.2986

In [35]:
ACTIONS = [Action.HIT, Action.STAND, Action.DOUBLE]

def test_cond(score_p, score_d, n):
    def cond(h, d):
        return h.score == score_p and d.score == score_d
    strats = gen_cond_strategies(strat_simple, cond, ACTIONS)
    hand = make_hand([8, score_p-8]) # This will only work for 10 through 18 
    sims = run_n_sim_trials_from_state(strats, hand, make_hand([score_d]), n)
    print(summarize_totals(sims))
    return cond, summarize_totals(sims)

test_cond(10, 6, 200)

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0          62   12    0        126        0.640
<Action.HIT: 2>      60           0   13  127          0        0.335
<Action.STAND: 1>   110           0    0   90          0       -0.100


(<function __main__.test_cond.<locals>.cond(h, d)>,
                     len                                  mean_outcome
 outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
 strategy                                                             
 <Action.DOUBLE: 3>    0          62   12    0        126        0.640
 <Action.HIT: 2>      60           0   13  127          0        0.335
 <Action.STAND: 1>   110           0    0   90          0       -0.100)

In [36]:
def find_winning_action(score_p, score_d, n):
    cond, summary = test_cond(score_p, score_d, n)
    outcomes = summary['mean_outcome']
    winner = outcomes[outcomes==outcomes.max()].index[0]
    winning_act = [a for a in ACTIONS if repr(a)==winner][0]
    return (cond, winning_act)

find_winning_action(10, 6, 200)

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0          64   12    0        124        0.600
<Action.HIT: 2>      67           0   15  118          0        0.255
<Action.STAND: 1>   113           0    0   87          0       -0.130


(<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>)

In [37]:
conds = [find_winning_action(p, d, 1000) for p in range(10, 19) for d in range(1, 11)]

                    len           mean_outcome
outcome_name       LOSE PUSH  WIN             
strategy                                      
<Action.DOUBLE: 3>  562  108  330       -0.232
<Action.HIT: 2>     549   94  357       -0.192
<Action.STAND: 1>   539   90  371       -0.168
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         368   73    0        559        0.382
<Action.HIT: 2>     372           0  102  526          0        0.154
<Action.STAND: 1>   651           0    0  349          0       -0.302
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         345   77    0        578        0.466
<Action.HIT: 2>   

                    len           mean_outcome
outcome_name       LOSE PUSH  WIN             
strategy                                      
<Action.DOUBLE: 3>  736   49  215       -0.521
<Action.HIT: 2>     715   56  229       -0.486
<Action.STAND: 1>   721   64  215       -0.506
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         607   49    0        344       -0.526
<Action.HIT: 2>     582           0   66  352          0       -0.230
<Action.STAND: 1>   662           0    0  338          0       -0.324
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         577   47    0        376       -0.402
<Action.HIT: 2>   

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         652   42    0        306       -0.692
<Action.HIT: 2>     657           0   47  296          0       -0.361
<Action.STAND: 1>   623           0    0  377          0       -0.246
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         633   58    0        309       -0.648
<Action.HIT: 2>     660           0   37  303          0       -0.357
<Action.STAND: 1>   605           0    0  395          0       -0.210
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy            

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         716   33    0        251       -0.930
<Action.HIT: 2>     689           0   61  250          0       -0.439
<Action.STAND: 1>   599           0    0  401          0       -0.198
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         697   45    0        258       -0.878
<Action.HIT: 2>     735           0   39  226          0       -0.509
<Action.STAND: 1>   611           0    0  389          0       -0.222
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy            

                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         789   30    0        181       -1.216
<Action.HIT: 2>     785           0   29  186          0       -0.599
<Action.STAND: 1>   324           0  139  537          0        0.213
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy                                                             
<Action.DOUBLE: 3>    0         809   23    0        168       -1.282
<Action.HIT: 2>     794           0   34  172          0       -0.622
<Action.STAND: 1>   329           0  106  565          0        0.236
                    len                                  mean_outcome
outcome_name       LOSE LOSE_DOUBLE PUSH  WIN WIN_DOUBLE             
strategy            

In [38]:
conds

[(<function __main__.test_cond.<locals>.cond(h, d)>, <Action.STAND: 1>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.HIT: 2>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.cond(h, d)>, <Action.DOUBLE: 3>),
 (<function __main__.test_cond.<locals>.co

In [39]:
strat_derived = generate_strat_conditional(strat_simple, conds)
strat_derived.name = 'strat_derived'

In [40]:
strat_derived2 = generate_strat_conditional(strat_derived, conds)
strat_derived2.name = 'strat_derived2'

In [41]:
sims = run_n_sim_trials([strat_simple, strat_derived, strat_derived2], 100*1000)
sims.head(10), summarize_totals(sims)

(         strategy hand_start  dealer_card       hand_end dealer_hand  \
 0          simple   [13, 13]            2       [13, 13]   [2, 6, 1]   
 1   strat_derived   [13, 13]            2       [13, 13]   [2, 6, 1]   
 2  strat_derived2   [13, 13]            2       [13, 13]   [2, 6, 1]   
 3          simple     [6, 2]            3     [6, 2, 13]  [3, 12, 6]   
 4   strat_derived     [6, 2]            3      [6, 2, 4]  [3, 12, 6]   
 5  strat_derived2     [6, 2]            3  [6, 2, 2, 11]  [3, 12, 6]   
 6          simple     [1, 4]            3         [1, 4]  [3, 11, 7]   
 7   strat_derived     [1, 4]            3         [1, 4]  [3, 11, 7]   
 8  strat_derived2     [1, 4]            3         [1, 4]  [3, 11, 7]   
 9          simple     [7, 8]            9     [7, 8, 10]     [9, 13]   
 
                   outcome  outcome_value outcome_name  
 0         HandOutcome.WIN            1.0          WIN  
 1         HandOutcome.WIN            1.0          WIN  
 2         HandOutcome.W